<a href="https://colab.research.google.com/github/sukhyun1017/pytorch-study/blob/main/pytorch_softmax_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F

In [4]:
z=torch.rand(3,5,requires_grad=True) #0과 1사이 숫자를 랜덤하게 생성

In [5]:
H=F.softmax(z,dim=0) #열의 합이 1/5개의 샘플에 대해 3개의 클래스로 분류
print(H)

tensor([[0.3895, 0.3696, 0.3538, 0.2640, 0.3422],
        [0.2836, 0.2824, 0.2722, 0.2874, 0.2930],
        [0.3269, 0.3480, 0.3741, 0.4486, 0.3648]], grad_fn=<SoftmaxBackward0>)


In [7]:
H = F.softmax(z, dim=1) #행의 합이 1/3개의 샘플에 대해 5개의 클래스로 분류
print(H)

tensor([[0.1879, 0.1727, 0.2522, 0.1356, 0.2517],
        [0.1656, 0.1598, 0.2349, 0.1787, 0.2609],
        [0.1452, 0.1498, 0.2456, 0.2122, 0.2471]], grad_fn=<SoftmaxBackward0>)


In [9]:
y = torch.randint(5, (3,1)) # 랜덤하게 정수생성 #5미만 정수 랜덤 생성
print(y)

tensor([[4],
        [4],
        [3]])


In [10]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(H) #zeros_like  tensor 를 사이즈로 받음
y_one_hot.scatter_(1, y, 1) # scatter(dim=1,y,채울 숫자)

tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.]])

In [11]:
cost = (y_one_hot * -torch.log(H)).sum(dim=1).mean()
print(cost)

tensor(1.4245, grad_fn=<MeanBackward0>)


In [12]:
torch.log(H)

tensor([[-1.6720, -1.7560, -1.3777, -1.9983, -1.3797],
        [-1.7979, -1.8338, -1.4484, -1.7222, -1.3435],
        [-1.9294, -1.8982, -1.4040, -1.5504, -1.3978]], grad_fn=<LogBackward0>)

In [13]:
F.log_softmax(z,dim=1)

tensor([[-1.6720, -1.7560, -1.3777, -1.9983, -1.3797],
        [-1.7979, -1.8338, -1.4484, -1.7222, -1.3435],
        [-1.9294, -1.8982, -1.4040, -1.5504, -1.3978]],
       grad_fn=<LogSoftmaxBackward0>)

In [15]:
(y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4245, grad_fn=<MeanBackward0>)

In [22]:
F.nll_loss(F.log_softmax(z, dim=1),y.squeeze(1)) #negative log likelihood

tensor(1.4245, grad_fn=<NllLossBackward0>)

In [23]:
F.cross_entropy(z, y.squeeze(1))

tensor(1.4245, grad_fn=<NllLossBackward0>)

소프트맥스 구현하기

In [25]:
import torch.nn.functional as F
import torch.optim as optim

In [24]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #3개의 클래스
x_train = torch.FloatTensor(x_train) #32비트 실수 텐서
y_train = torch.LongTensor(y_train) #64비트 정수 텐서

In [26]:
y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [62]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1,3), requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [58]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    H = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(H)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [56]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1,3), requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.623000
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285073
Epoch 1000/1000 Cost: 0.248155


In [51]:
import torch.nn as nn
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4,3)
  
  def forward(self,x):
    return self.linear(x)

In [39]:
model =SoftmaxClassifierModel()

In [55]:
model =SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.167313
Epoch  100/1000 Cost: 0.661658
Epoch  200/1000 Cost: 0.572799
Epoch  300/1000 Cost: 0.517248
Epoch  400/1000 Cost: 0.472406
Epoch  500/1000 Cost: 0.432747
Epoch  600/1000 Cost: 0.395879
Epoch  700/1000 Cost: 0.360299
Epoch  800/1000 Cost: 0.324827
Epoch  900/1000 Cost: 0.288654
Epoch 1000/1000 Cost: 0.253726
